In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

caminhos = [
    "../files/apostila.pdf",
    "../files/LLM.pdf"
]

paginas = []

for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = recur_split.split_documents(paginas)

/Users/thalesmarega/Documents/thalesmarega/workspace-python/estudos-agents/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong poi

In [2]:
for i, doc in enumerate(documents):
    doc.metadata['source'] = doc.metadata['source'].replace('files/', '')
    doc.metadata['doc_id'] = i

In [3]:
documents[2].metadata

{'source': '../apostila.pdf', 'page': 1, 'doc_id': 2}

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embeddings_model = OpenAIEmbeddings()

diretorio = '../files/chroma_retrival_bd'

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=diretorio
)

### Semantic Search

In [8]:
pergunta = "O que é LLM?"
docs = vectordb.similarity_search(pergunta, k=3)
for doc in docs:
    print(doc.page_content)
    print(f"========{doc.metadata}\n")

. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
========{'doc_id': 141, 'page': 7, 'source': 'LLM.pdf'}

PARTE  1 Introdução   Definição de LLM (tradução livre: grandes modelos de linguagem) LLMs são sistemas de IA desenvolvidos para processar e analisar enormes quantidades de dados de linguagem natural e, em seguida, usar essas informações para gerar respostas às solicitações dos usuários
========{'doc_id': 101, 'page': 1, 'source': 'LLM.pdf'}

. Conclusão e diretrizes gerais Em última análise, cada organização terá desafios únicos a superar, e não existe uma abordagem única para os LLMs. À medida que o mundo se torna mais orientado a dados, tudo, incluindo os LLMs, dependerá de uma base sólida de dados. Os LLMs são ferramentas incríveis, ma

### Max Margina Relevance

In [7]:
pergunta = "O que é LLM?"
docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=10)
for doc in docs:
    print(doc.page_content)
    print(f"========{doc.metadata}\n")

. Se você quiser se aprofundar um pouco mais e expandir seus conhecimentos e compreensão dos fundamentos dos LLMs, recomendamos conferir nosso curso sobre LLMs. Você aprenderá como desenvolver aplicativos prontos para produção com LLMs e se aprofundará na teoria por trás dos modelos de fundação
========{'doc_id': 141, 'page': 7, 'source': '../LLM.pdf'}

PARTE  1 Introdução   Definição de LLM (tradução livre: grandes modelos de linguagem) LLMs são sistemas de IA desenvolvidos para processar e analisar enormes quantidades de dados de linguagem natural e, em seguida, usar essas informações para gerar respostas às solicitações dos usuários
========{'doc_id': 101, 'page': 1, 'source': '../LLM.pdf'}

. Conclusão e diretrizes gerais Em última análise, cada organização terá desafios únicos a superar, e não existe uma abordagem única para os LLMs. À medida que o mundo se torna mais orientado a dados, tudo, incluindo os LLMs, dependerá de uma base sólida de dados. Os LLMs são ferramentas incríve

### FIltragem

In [10]:
pergunta = "O que a apostila de LLM fala sobre a OpenAI e o ChatGPT?"

docs = vectordb.similarity_search(
    pergunta,
    k=3,
    filter={"source": "LLM.pdf"}
)
for doc in docs:
    print(doc.page_content)
    print(f"========{doc.metadata}\n")

In [11]:
pergunta = 'O que a apostila de LLM fala sobre a OpenAI e o ChatGPT?'

docs = vectordb.similarity_search(
    pergunta, 
    k=3,
    filter={'$and':
            [{'source': {'$in': ['LLM.pdf']}},
            {'page': {'$in': [3, 4, 5, 6]}}],
            })
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')

### LLM Aided Retrieval

In [15]:
from langchain_openai.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.schema import AttributeInfo

metadata_info = [
    AttributeInfo(
        name='source',
        description='Nome da apostila de onde o texto original foi retirado. Pode ser "apostila.pdf" ou "LLM.pdf".',
        type='string'
    ),
    AttributeInfo(
        name='page',
        description='A página da apostila de onde o texto foi extraído. Número da página.',
        type='integer'
    ),
]

document_description = 'Apostilas de informações'
llm = OpenAI()
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_description,
    metadata_info,
    verbose=True
)

pergunta = 'O que a apostila de LLM fala sobre a OpenAI, ChatGPT e Hugging Face?'

docs = retriever.get_relevant_documents(pergunta)
for doc in docs:
    print(doc.page_content)
    print(f'==========={doc.metadata}\n\n')